# 🧼  <span style="color:#f6f794"> Data Analysis </span>
___

#### 📚 <span style="color:#f6f794"> Importing libraries and setting displays </span>

In [1]:
import pandas as pd
import numpy as np
import os
import re
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy.stats import ks_2samp


import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
pd.set_option('display.max_rows', 100) 
pd.set_option('display.max_columns', None) 

# pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', None)     # Show all rows
# pd.set_option('display.max_colwidth', None) # Show entire content of each column
pd.set_option('display.width', None)        # No limit on display width
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping

#### 📂 <span style="color:#f6f794"> Opening files and create dataframes of them </span>

In [4]:
df_clean = pd.read_csv("clean_data_states_daily.csv",index_col=0)
df_clean.sample(5)

,state,state_name,positive,probable_cases,negative,total_test_results,hospitalized_currently,hospitalized_cumulative,in_icu_currently,in_icu_cumulative,recovered,last_update_et,death,death_confirmed,death_probable,positive_increase,total_test_results_increase,death_increase,hospitalized_increase
date,,,,,,,,,,,,,,,,,,,
2020-06-04,DE,Delaware,9746.0,NaN,56163.0,79033.0,142.0,NaN,NaN,NaN,5562.0,2020-06-03 18:00:00,483.0,425.0,58.0,34,1232,8,0
2020-11-13,UT,Utah,145789.0,NaN,1018942.0,1567347.0,492.0,6591.0,184.0,1294.0,100892.0,2020-11-13 13:34:00,701.0,NaN,NaN,2150,15697,14,104
2020-08-02,OR,Oregon,18817.0,NaN,390799.0,644785.0,208.0,1607.0,65.0,NaN,3872.0,2020-08-01 03:01:00,325.0,NaN,NaN,325,9366,3,0
2020-09-20,AZ,Arizona,214018.0,4317.0,1169906.0,2035326.0,472.0,21876.0,120.0,NaN,NaN,2020-09-20 00:00:00,5476.0,5184.0,292.0,467,8384,9,-6
2020-04-06,MO,Missouri,2722.0,NaN,27113.0,31954.0,439.0,NaN,NaN,NaN,NaN,2020-04-06 16:28:00,39.0,NaN,NaN,355,1103,5,0
